## 필요 모듈 임포트

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
from collections import Counter
import re

## 데이터 다운로드

In [ ]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

print("Train data head:")
display(train_data.head())

print("\nTest data head:")
display(test_data.head())

Train data head:


,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...



Test data head:


,idx,text
0,t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
1,t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
2,t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
3,t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
4,t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...


In [ ]:
conversations_df = pd.read_csv('conversations.csv')
display(conversations_df.head())

,idx,class,conversation
0,0,일반 대화,우리 이번 여름에 울릉도 갈까?\n울릉도 좋지! 근데 왜 갑분 울릉도?\n울릉도에서...
1,1,일반 대화,요즘 마라탕 덕후됨 ㅠㅠ 너무 맛있음.\n그니까 ㅠㅠ 마라탕 넘맛! 요즘 사람들 마...
2,2,일반 대화,저번에 식사 자리에서 보니까 너 전통주에 대해서 꽤 잘 알더라? 완전 고수 같았어....
3,3,일반 대화,"야, 요즘 전통주 구독 서비스가 있다던데 알아?\n당연히 알지! 매달 다른 전통주가..."
4,4,일반 대화,"최근 일식집에 한 번 갔다 왔는데, 마제소바라는 요리가 있길래 처음 먹어봤는데 완전..."


In [ ]:
normal_conversations_df = conversations_df[conversations_df['class'] == '일반 대화'].copy()
display(normal_conversations_df.head())

,idx,class,conversation
0,0,일반 대화,우리 이번 여름에 울릉도 갈까?\n울릉도 좋지! 근데 왜 갑분 울릉도?\n울릉도에서...
1,1,일반 대화,요즘 마라탕 덕후됨 ㅠㅠ 너무 맛있음.\n그니까 ㅠㅠ 마라탕 넘맛! 요즘 사람들 마...
2,2,일반 대화,저번에 식사 자리에서 보니까 너 전통주에 대해서 꽤 잘 알더라? 완전 고수 같았어....
3,3,일반 대화,"야, 요즘 전통주 구독 서비스가 있다던데 알아?\n당연히 알지! 매달 다른 전통주가..."
4,4,일반 대화,"최근 일식집에 한 번 갔다 왔는데, 마제소바라는 요리가 있길래 처음 먹어봤는데 완전..."


In [ ]:
train_data = train_data[train_data['class'] != '일반 대화']
display(train_data.head())

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [ ]:
merged_df = pd.concat([normal_conversations_df, train_data], ignore_index=True)
display(merged_df.head())
display(merged_df['class'].value_counts())

,idx,class,conversation
0,0,일반 대화,우리 이번 여름에 울릉도 갈까?\n울릉도 좋지! 근데 왜 갑분 울릉도?\n울릉도에서...
1,1,일반 대화,요즘 마라탕 덕후됨 ㅠㅠ 너무 맛있음.\n그니까 ㅠㅠ 마라탕 넘맛! 요즘 사람들 마...
2,2,일반 대화,저번에 식사 자리에서 보니까 너 전통주에 대해서 꽤 잘 알더라? 완전 고수 같았어....
3,3,일반 대화,"야, 요즘 전통주 구독 서비스가 있다던데 알아?\n당연히 알지! 매달 다른 전통주가..."
4,4,일반 대화,"최근 일식집에 한 번 갔다 왔는데, 마제소바라는 요리가 있길래 처음 먹어봤는데 완전..."


,count
class,
일반 대화,156981
기타 괴롭힘 대화,1094
갈취 대화,981
직장 내 괴롭힘 대화,979
협박 대화,896


In [ ]:
merged_df = merged_df.reset_index(drop=True)
display(merged_df.head())
display(merged_df['class'].value_counts())

,idx,class,conversation
0,0,일반 대화,우리 이번 여름에 울릉도 갈까?\n울릉도 좋지! 근데 왜 갑분 울릉도?\n울릉도에서...
1,1,일반 대화,요즘 마라탕 덕후됨 ㅠㅠ 너무 맛있음.\n그니까 ㅠㅠ 마라탕 넘맛! 요즘 사람들 마...
2,2,일반 대화,저번에 식사 자리에서 보니까 너 전통주에 대해서 꽤 잘 알더라? 완전 고수 같았어....
3,3,일반 대화,"야, 요즘 전통주 구독 서비스가 있다던데 알아?\n당연히 알지! 매달 다른 전통주가..."
4,4,일반 대화,"최근 일식집에 한 번 갔다 왔는데, 마제소바라는 요리가 있길래 처음 먹어봤는데 완전..."


,count
class,
일반 대화,156981
기타 괴롭힘 대화,1094
갈취 대화,981
직장 내 괴롭힘 대화,979
협박 대화,896


In [ ]:
train_data = merged_df

In [ ]:
x_train = train_data['conversation']
y_train = train_data['class']

In [ ]:
# 불용어(stop_words) 목록 정의 (조사, 접속사 등)
# 일단은 작은 불용어 사전을 이용하여 빈도수 측정
stop_words = ['은', '는', '이', '가', '다', '을', '를', '에', '와', '과', '에서']

# 텍스트 전처리 및 키워드 추출 함수
def preprocess_korean_text(text):
    text = re.sub(r'[^가-힣\s]', '', text) # 한글과 공백만 남기기
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words and len(word) > 1] # 불용어 및 한 글자 단어 제거
    return filtered_words # 단어 리스트 반환

# 각 클래스별 키워드 빈도 분석
for cls in train_data['class'].unique():
    class_df = train_data[train_data['class'] == cls]
    all_text = ' '.join(class_df['conversation'])
    keywords = preprocess_korean_text(all_text)

    # 상위 10개 키워드 출력
    print(f"\n--- {cls}의 주요 키워드 ---")
    # Counter에 단어 리스트를 전달하여 빈도 계산
    print(Counter(keywords).most_common(10))


--- 일반 대화의 주요 키워드 ---
[('맞아', 206002), ('거야', 193960), ('있어', 187175), ('그럼', 133133), ('같아', 128831), ('어떤', 123193), ('그래', 121497), ('그래서', 121312), ('나도', 92408), ('그러면', 85333)]

--- 협박 대화의 주요 키워드 ---
[('내가', 923), ('제발', 397), ('지금', 337), ('진짜', 311), ('그냥', 241), ('니가', 241), ('그래', 228), ('죄송합니다', 222), ('아니', 219), ('그럼', 208)]

--- 기타 괴롭힘 대화의 주요 키워드 ---
[('내가', 647), ('진짜', 440), ('아니', 375), ('그냥', 314), ('그래', 245), ('지금', 233), ('아니야', 225), ('그렇게', 223), ('무슨', 222), ('니가', 218)]

--- 갈취 대화의 주요 키워드 ---
[('내가', 704), ('진짜', 400), ('그럼', 366), ('이거', 362), ('아니', 343), ('지금', 316), ('그래', 273), ('돈이', 255), ('없어', 241), ('그냥', 237)]

--- 직장 내 괴롭힘 대화의 주요 키워드 ---
[('죄송합니다', 840), ('내가', 535), ('아니', 389), ('제가', 386), ('지금', 324), ('그럼', 275), ('오늘', 243), ('부장님', 239), ('무슨', 217), ('그렇게', 199)]


## 전처리

In [ ]:
# 한글 전처리 함수
def preprocess_korean_text(text):
    # 불필요한 공백 제거
    text = re.sub(r'\s+', ' ', text)
    # 특수문자 정규화 (일부 보존)
    text = re.sub(r'[^\w\s\.\!\?\,\n]', ' ', text)
    # 연속된 문장부호 정리
    text = re.sub(r'[\.]{2,}', '.', text)
    text = re.sub(r'[\!]{2,}', '!', text)
    text = re.sub(r'[\?]{2,}', '?', text)
    # 줄바꿈을 공백으로 변경
    text = text.replace('\n', ' ')
    # 앞뒤 공백 제거
    text = text.strip()
    return text

## 모델

In [ ]:
# 데이터셋 클래스
class ThreatDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=300):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        text = preprocess_korean_text(text)

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

In [ ]:
class TestDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=300):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        text = preprocess_korean_text(text)

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

### 파인튜닝

In [ ]:
def train_model(model, train_loader, val_loader, device, epochs=3, max_steps=None):
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=2e-5)

    best_val_f1 = 0
    global_step = 0

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs} 시작")

        # 훈련
        model.train()
        train_loss = 0
        step = 0

        for batch in train_loader:
            optimizer.zero_grad()

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            step += 1
            global_step += 1

            if step % 100 == 0:
                print(f"Step {step}, Loss: {loss.item():.4f}")

            if max_steps is not None and global_step >= max_steps:
                print(f"Max steps ({max_steps}) reached. Stopping training.")
                break # Break from the inner loop

        # 검증
        model.eval()
        val_predictions = []
        val_labels = []

        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                predictions = torch.argmax(outputs.logits, dim=-1)

                val_predictions.extend(predictions.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        val_acc = accuracy_score(val_labels, val_predictions)
        val_f1 = f1_score(val_labels, val_predictions, average='weighted')

        print(f"Epoch {epoch+1} 완료")
        print(f"Train Loss: {train_loss/len(train_loader):.4f}")
        print(f"Val Accuracy: {val_acc:.4f}, Val F1: {val_f1:.4f}")

        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), 'best_model.pt')
            print("최고 성능 모델 저장")

        if max_steps is not None and global_step >= max_steps:
            break # Break from the outer loop

    return model

### 모델 학습 및 제출

In [ ]:
def main():
    print("한국어 위협 대화 분류 모델 시작")

    # 디바이스 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")

    # 데이터 로드
    print("데이터 로드 중...")
    train_df = pd.read_csv('train.csv')
    test_df = pd.read_csv('test.csv')

    # 이진 분류를 위한 데이터 준비 (conversations.csv의 일반 대화와 train.csv 병합)
    print("이진 분류를 위한 데이터 준비 중...")
    conversations_df = pd.read_csv('conversations.csv')
    normal_conversations_df = conversations_df[conversations_df['class'] == '일반 대화'].copy()
    train_df_no_normal = train_df[train_df['class'] != '일반 대화'].copy()
    merged_df = pd.concat([normal_conversations_df, train_df_no_normal], ignore_index=True)
    train_df = merged_df.reset_index(drop=True) # Update train_df to the merged data

    # '일반 대화'는 1, 나머지는 0으로 매핑
    train_df['binary_label'] = train_df['class'].apply(lambda x: 1 if x == '일반 대화' else 0)

    # 이진 분류 데이터 분할 (전체 병합 데이터 사용)
    X_train_binary, X_val_binary, y_train_binary, y_val_binary = train_test_split(
        train_df['conversation'].values,
        train_df['binary_label'].values,
        test_size=0.2,
        random_state=42,
        stratify=train_df['binary_label'].values
    )

    print(f"이진 분류 훈련 데이터: {len(X_train_binary)}개")
    print(f"이진 분류 검증 데이터: {len(X_val_binary)}개")

    # 클래스 매핑 (세부 분류용)
    class_mapping = {
        '협박 대화': 0,
        '갈취 대화': 1,
        '직장 내 괴롭힘 대화': 2,
        '기타 괴롭힘 대화': 3
    }
    reverse_class_mapping = {v: k for k, v in class_mapping.items()}


    # 세부 분류를 위한 데이터 필터링 (일반 대화 제외 - 원래 train.csv의 비일반 대화 사용)
    train_df_detailed = train_df[train_df['class'] != '일반 대화'].copy()
    train_df_detailed = train_df_detailed[train_df_detailed['class'].isin(class_mapping.keys())].copy()
    train_df_detailed['detailed_label'] = train_df_detailed['class'].map(class_mapping)


    # 세부 분류 데이터 분할
    X_train_detailed, X_val_detailed, y_train_detailed, y_val_detailed = train_test_split(
        train_df_detailed['conversation'].values,
        train_df_detailed['detailed_label'].values,
        test_size=0.2,
        random_state=42,
        stratify=train_df_detailed['detailed_label'].values
    )

    print(f"세부 분류 훈련 데이터: {len(X_train_detailed)}개")
    print(f"세부 분류 검증 데이터: {len(X_val_detailed)}개")


    # 토크나이저 및 모델 로드 (이진 분류 및 세부 분류 모두에 동일한 토크나이저 사용)
    print("토크나이저 로드 중...")
    tokenizer = BertTokenizer.from_pretrained('beomi/kcbert-base')

    # 이진 분류 모델 정의
    print("이진 분류 모델 정의 중...")
    binary_model = BertForSequenceClassification.from_pretrained('beomi/kcbert-base', num_labels=2)

    # 세부 분류 모델 정의
    print("세부 분류 모델 정의 중...")
    detailed_model = BertForSequenceClassification.from_pretrained('beomi/kcbert-base', num_labels=4)


    # 이진 분류 데이터셋 및 데이터로더 생성
    train_dataset_binary = ThreatDataset(X_train_binary, y_train_binary, tokenizer)
    val_dataset_binary = ThreatDataset(X_val_binary, y_val_binary, tokenizer)
    test_dataset = TestDataset(test_df['text'].values, tokenizer) # Test set will be handled after binary prediction

    train_loader_binary = DataLoader(train_dataset_binary, batch_size=16, shuffle=True)
    val_loader_binary = DataLoader(val_dataset_binary, batch_size=16, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


    # 세부 분류 데이터셋 및 데이터로더 생성
    train_dataset_detailed = ThreatDataset(X_train_detailed, y_train_detailed, tokenizer)
    val_dataset_detailed = ThreatDataset(X_val_detailed, y_val_detailed, tokenizer)


    train_loader_detailed = DataLoader(train_dataset_detailed, batch_size=16, shuffle=True)
    val_loader_detailed = DataLoader(val_dataset_detailed, batch_size=16, shuffle=False)


    # 이진 분류 모델 훈련
    print("이진 분류 모델 훈련 시작...")
    binary_model = train_model(binary_model, train_loader_binary, val_loader_binary, device, epochs=1, max_steps=1000) # Added max_steps
    torch.save(binary_model.state_dict(), 'binary_model.pt') # 이진 분류 모델 저장

    # 세부 분류 모델 훈련
    print("세부 분류 모델 훈련 시작...")
    detailed_model = train_model(detailed_model, train_loader_detailed, val_loader_detailed, device, epochs=3) # Epochs remain 3
    torch.save(detailed_model.state_dict(), 'detailed_model.pt') # 세부 분류 모델 저장



In [ ]:
main()

한국어 위협 대화 분류 모델 시작
Device: cuda
데이터 로드 중...
이진 분류를 위한 데이터 준비 중...
이진 분류 훈련 데이터: 128744개
이진 분류 검증 데이터: 32187개
세부 분류 훈련 데이터: 3160개
세부 분류 검증 데이터: 790개
토크나이저 로드 중...
이진 분류 모델 정의 중...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


세부 분류 모델 정의 중...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


이진 분류 모델 훈련 시작...
Epoch 1/1 시작
Step 100, Loss: 0.0020
Step 200, Loss: 0.0010
Step 300, Loss: 0.0005
Step 400, Loss: 0.0003
Step 500, Loss: 0.0002
Step 600, Loss: 0.0001
Step 700, Loss: 0.0001
Step 800, Loss: 0.0001
Step 900, Loss: 0.0001
Step 1000, Loss: 0.0001
Max steps (1000) reached. Stopping training.
Epoch 1 완료
Train Loss: 0.0009
Val Accuracy: 0.9998, Val F1: 0.9998
최고 성능 모델 저장
세부 분류 모델 훈련 시작...
Epoch 1/3 시작
Step 100, Loss: 0.3693
Epoch 1 완료
Train Loss: 0.5518
Val Accuracy: 0.8924, Val F1: 0.8926
최고 성능 모델 저장
Epoch 2/3 시작
Step 100, Loss: 0.3549
Epoch 2 완료
Train Loss: 0.1891
Val Accuracy: 0.8722, Val F1: 0.8741
Epoch 3/3 시작
Step 100, Loss: 0.0364
Epoch 3 완료
Train Loss: 0.0902
Val Accuracy: 0.9025, Val F1: 0.9025
최고 성능 모델 저장


In [ ]:
print("한국어 위협 대화 분류 모델 예측 및 submission 파일 생성 시작")

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# 데이터 로드
print("테스트 데이터 로드 중...")
test_df = pd.read_csv('test.csv')

# 토크나이저 로드
print("토크나이저 로드 중...")
tokenizer = BertTokenizer.from_pretrained('beomi/kcbert-base')

# 이진 분류 모델 정의 및 로드
print("이진 분류 모델 로드 중...")
binary_model = BertForSequenceClassification.from_pretrained('beomi/kcbert-base', num_labels=2) # 2 classes for binary
binary_model.load_state_dict(torch.load('binary_model.pt', map_location=device))
binary_model.to(device)
binary_model.eval()

# 세부 분류 모델 정의 및 로드
print("세부 분류 모델 로드 중...")
detailed_model = BertForSequenceClassification.from_pretrained('beomi/kcbert-base', num_labels=4) # 4 classes for detailed
detailed_model.load_state_dict(torch.load('detailed_model.pt', map_location=device))
detailed_model.to(device)
detailed_model.eval()

# 클래스 매핑 (세부 분류용 - 사용자 정의)
class_mapping_detailed = {
    '협박 대화': 0,
    '갈취 대화': 1,
    '직장 내 괴롭힘 대화': 2,
    '기타 괴롭힘 대화': 3
}
# reverse_class_mapping_detailed = {v: k for k, v in class_mapping_detailed.items()} # Reverse mapping is not needed for submission


# 테스트 데이터셋 및 데이터로더 생성
test_dataset = TestDataset(test_df['text'].values, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# 테스트 데이터 예측
print("테스트 데이터 예측 중...")
test_predictions = []

with torch.no_grad():
    for i, batch in enumerate(test_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        # 이진 분류 예측
        binary_outputs = binary_model(input_ids=input_ids, attention_mask=attention_mask)
        binary_predictions = torch.argmax(binary_outputs.logits, dim=-1)

        # '일반 대화'로 분류된 경우
        for j, pred in enumerate(binary_predictions):
            if pred.item() == 1: # 이진 분류에서 일반 대화 (라벨 1)
                test_predictions.append(4) # 일반 대화는 4로 매핑
            else: # 비일반 대화, 세부 분류 필요 (이진 분류 라벨 0)
                # 해당 샘플만 추출하여 세부 분류 모델에 전달
                single_input_ids = input_ids[j].unsqueeze(0)
                single_attention_mask = attention_mask[j].unsqueeze(0)

                detailed_outputs = detailed_model(input_ids=single_input_ids, attention_mask=single_attention_mask)
                detailed_prediction = torch.argmax(detailed_outputs.logits, dim=-1).item()
                # 세부 분류 결과에 해당하는 숫자 라벨 사용 (0, 1, 2, 3)
                test_predictions.append(detailed_prediction)


# 결과 종합 및 저장
print("결과 종합 및 submission.csv 저장 중...")
submission_df = pd.DataFrame({'idx': test_df['idx'], 'class': test_predictions})
submission_df.to_csv('submission.csv', index=False)

print("submission.csv 파일이 사용자 정의 라벨링으로 성공적으로 생성되었습니다.")

한국어 위협 대화 분류 모델 예측 및 submission 파일 생성 시작
Device: cuda
테스트 데이터 로드 중...
토크나이저 로드 중...
이진 분류 모델 로드 중...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


세부 분류 모델 로드 중...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


테스트 데이터 예측 중...
결과 종합 및 submission.csv 저장 중...
submission.csv 파일이 사용자 정의 라벨링으로 성공적으로 생성되었습니다.


In [ ]:
binary_logits = []
detailed_logits = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        binary_outputs = binary_model(input_ids=input_ids, attention_mask=attention_mask)
        binary_logits.extend(binary_outputs.logits.cpu().numpy())

        detailed_outputs = detailed_model(input_ids=input_ids, attention_mask=attention_mask)
        detailed_logits.extend(detailed_outputs.logits.cpu().numpy())

binary_logits = np.array(binary_logits)
detailed_logits = np.array(detailed_logits)

print("Binary logits shape:", binary_logits.shape)
print("Detailed logits shape:", detailed_logits.shape)

Binary logits shape: (500, 2)
Detailed logits shape: (500, 4)


이진및 상세 로짓에 소프트맥스를 적용하여 확률을 얻고, 이를 결합하여 최종 5개 클래스 확률 분포를 구합다


In [ ]:
binary_probabilities = torch.softmax(torch.tensor(binary_logits), dim=1).numpy()

detailed_probabilities = torch.softmax(torch.tensor(detailed_logits), dim=1).numpy()


final_probabilities = np.zeros((len(test_df), 5))

final_probabilities[:, 4] = binary_probabilities[:, 1]

non_normal_probability = binary_probabilities[:, 0]

final_probabilities[:, 0:4] = non_normal_probability[:, np.newaxis] * detailed_probabilities

print("Final probabilities shape:", final_probabilities.shape)
display(final_probabilities[:5])

Final probabilities shape: (500, 5)


array([[8.14117584e-03, 9.83124673e-01, 1.00808416e-03, 7.50508253e-03,
        2.20973074e-04],
       [8.41043424e-04, 4.28229687e-04, 9.97935057e-01, 5.45065501e-04,
        2.50585406e-04],
       [2.42906809e-01, 8.79035145e-03, 7.21153975e-01, 2.69612670e-02,
        1.87591068e-04],
       [1.18093742e-02, 2.96228454e-02, 6.82483800e-03, 9.11077738e-01,
        4.06652242e-02],
       [9.74044390e-03, 5.11627756e-02, 4.99298424e-03, 9.33888495e-01,
        2.15287437e-04]])

최종 확률 분포와 목표 샘플 수(각 클래스 100개)를 고려하여 예측 결과를 조정합니다.


In [ ]:
target_samples_per_class = 100
num_classes = 5
total_samples = len(test_df)

final_predictions_adjusted = [-1] * total_samples

assigned_indices = set()

for class_label in range(num_classes):
    if class_label < 4:
        sorted_indices = np.argsort(final_probabilities[:, class_label])[::-1]
    else:
        sorted_indices = np.argsort(final_probabilities[:, 4])[::-1]

    samples_assigned_to_class = 0
    for idx in sorted_indices:
        if idx not in assigned_indices:
            final_predictions_adjusted[idx] = class_label
            assigned_indices.add(idx)
            samples_assigned_to_class += 1

        if samples_assigned_to_class >= target_samples_per_class:
            break

if len(assigned_indices) < total_samples:
    print(f"Warning: {total_samples - len(assigned_indices)} samples remain unassigned. Assigning based on highest remaining probability.")
    unassigned_indices = [i for i in range(total_samples) if i not in assigned_indices]
    for idx in unassigned_indices:

        pass

In [ ]:
submission_df_adjusted = pd.DataFrame({
    'idx': test_df['idx'],
    'class': final_predictions_adjusted
})

submission_df_adjusted.to_csv('submission.csv', index=False)

print("submission.csv file with adjusted predictions has been created.")

submission.csv file with adjusted predictions has been created.
